In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
import numpy as np

In [3]:
if torch.cuda.is_available():  
    DEVICE = "cuda:0"
    print('Using cuda!')
else:
    DEVICE = "cpu"

Using cuda!


In [4]:
transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

train_dir = './ImageEmotionDatasetMid/train'
train_dataset = datasets.ImageFolder(train_dir, transform=transform)

train_dataset.classes

['amusement', 'awe', 'contentment', 'excitement', 'sadness']

In [5]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 3, 5)  # in channel: 3, out channel: 3, kernel size: 5, output size: 28
        self.pool = nn.MaxPool2d(2, 2)  # size 14
        self.conv2 = nn.Conv2d(3, 5, 5)  # in: 6, out: 5, kernel: 5, output size: 10
        # self.pool again, size 5
        self.fc1 = nn.Linear(5 * 5 * 5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(32, 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(-1, 5 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
        
net = Net().to(DEVICE)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
epochs = 25

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # stats
        running_loss += loss.item()
    
    # end of epoch
    print(f'Epoch {epoch} - loss: {running_loss}')

print('Finished training')

Epoch 0 - loss: 81.24501085281372
Epoch 1 - loss: 79.14366602897644
Epoch 2 - loss: 78.50784718990326
Epoch 3 - loss: 77.91664469242096
Epoch 4 - loss: 75.51811003684998
Epoch 5 - loss: 76.41338539123535
Epoch 6 - loss: 72.85005503892899
Epoch 7 - loss: 70.84112095832825
Epoch 8 - loss: 68.92489749193192


In [ ]:
save_dir = './test_model.pth'
torch.save(net.state_dict(), save_dir)

In [ ]:
test_dir = './ImageEmotionDatasetMid/test'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

test_dataset.classes

In [ ]:
# copied this from the sample lol
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
dataiter = iter(test_loader)

images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
print('Ground Truth: ', ' '.join('%5s' % test_dataset.classes[labels[j]] for j in range(4)))

In [ ]:
with torch.no_grad():
    outputs = net(images.to(DEVICE))
    _, predicted = torch.max(outputs, 1)
    print('Predicted: ', ' '.join('%5s' % test_dataset.classes[predicted[j]] for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on all test images: %d %%' % (100 * correct / total))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on all train images: %d %%' % (100 * correct / total))

lmao completely overfitted